In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from tensorflow import keras
from sklearn import preprocessing
import keras.backend as K

#precision & recall by: https://github.com/keras-team/keras/issues/5400
#Code Source: https://cloud.google.com/blog/products/gcp/intro-to-text-classification-with-keras-automatically-tagging-stack-overflow-posts

trainDF = pd.concat([pd.read_pickle('../../data/raw_data/MasterData_2015.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2014.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2013.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2012.pkl.gz')])
                                   
trainDF = trainDF[trainDF.TEXT.notna() & trainDF.NTEECC.notna()]
trainDF['text'] = trainDF['TEXT'].astype(str)
trainDF['label'] = trainDF['NTEE'].astype(str)

trainDF = trainDF.drop(['NTEE', 'NTEECC', 'IRS_URL', 'TEXT'], axis=1)
trainDF = trainDF[~ (trainDF.label == 'nan')]

Using TensorFlow backend.


In [2]:
print("Total len: ", len(trainDF))
print(trainDF['label'].unique())

#test_df = pd.DataFrame(trainDF['label'].unique())
#test_df.to_csv("../../data/classification_results/results_nteecc/list_of_classes.csv")

#total classes: 1327

Total len:  464303
['E' 'T' 'C' 'X' 'N' 'A' 'L' 'P' 'K' 'O' 'B' 'D' 'S' 'W' 'F' 'M' 'G' 'Q'
 'I' 'H' 'U' 'Y' 'J' 'Z' 'R' 'V']


In [3]:
import statistics

counts = trainDF['label'].value_counts().sort_index().to_frame()
counts['category'] = counts.index
counts['train_sample']=(counts['label']/2).astype(int)
    
train_df, test_df = np.split(trainDF, [int(.7*len(trainDF))])

#tf.logging.set_verbosity(tf.logging.ERROR)

train_posts = train_df['text']
train_tags = train_df['label']
test_posts = test_df['text']
test_tags = test_df['label']
vocab_size = 1000
#precision & recall by: Other text preprocessing: https://keras.io/preprocessing/text/ 
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = preprocessing.LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
    
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.
     Only computes a batch-wise average of recall.
     Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

num_labels=len(train_df['label'].drop_duplicates())
batch_size = 500
epochs = 40

model = keras.Sequential()
model.add(keras.layers.Dense(512, input_shape=(vocab_size,)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(num_labels))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              #metrics=['accuracy'],
             metrics=['acc', precision, recall])

history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_split=0.1)
    
score = model.evaluate(x_test, y_test, 
                   batch_size=batch_size, verbose=1)


print(result)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 292510 samples, validate on 32502 samples
Epoch 1/40
292510/292510 [==============================] - 21s 71us/step - loss: 1.5260 - acc: 0.5946 - precision: 0.7675 - recall: 0.4086 - val_loss: 1.3169 - val_acc: 0.6342 - val_precision: 0.7917 - val_recall: 0.4838
Epoch 2/40
292510/292510 [==============================] - 20s 68us/step - loss: 1.2752 - acc: 0.6454 - precision: 0.7931 - recall: 0.5059 - val_loss: 1.2249 - val_acc: 0.6554 - val_precision: 0.7976 - val_recall: 0.5218
Epoch 3/40
292510/292510 [==============================] - 20s 67us/step - loss: 1.1764 - acc: 0.6685 - precision: 0.8082 - recall: 0.5400 - val_loss: 1.1497 - val_acc: 0.6771 - val_precision: 0.8137 - val_recall: 0.5474
Epoch 4/40
292510/292510 [==============================] - 20s 68us/step - loss: 1.0847 - acc: 0.6937 - precision: 0.8261 - recall: 0.5724 - val_loss: 1.0828 - val_acc: 0.6969 - val_precision: 0.8325 - val_rec

292510/292510 [==============================] - 19s 66us/step - loss: 0.2073 - acc: 0.9462 - precision: 0.9688 - recall: 0.9326 - val_loss: 0.6219 - val_acc: 0.8869 - val_precision: 0.9116 - val_recall: 0.8722
Epoch 37/40
292510/292510 [==============================] - 20s 67us/step - loss: 0.2034 - acc: 0.9474 - precision: 0.9693 - recall: 0.9341 - val_loss: 0.6253 - val_acc: 0.8863 - val_precision: 0.9117 - val_recall: 0.8719
Epoch 38/40
292510/292510 [==============================] - 20s 68us/step - loss: 0.2035 - acc: 0.9475 - precision: 0.9693 - recall: 0.9347 - val_loss: 0.6230 - val_acc: 0.8868 - val_precision: 0.9124 - val_recall: 0.8746
Epoch 39/40
292510/292510 [==============================] - 20s 68us/step - loss: 0.2014 - acc: 0.9481 - precision: 0.9696 - recall: 0.9357 - val_loss: 0.6178 - val_acc: 0.8888 - val_precision: 0.9134 - val_recall: 0.8740
Epoch 40/40
139291/139291 [==============================] - 5s 34us/step


NameError: name 'result' is not defined

In [4]:
#acc till 85: increases by 2% every epoch
#acc 85 - 95: increases by 1% each epoch
# 95 - 96: 4 epochs
# 96 - 97: >18 epochs

from tqdm import tqdm_notebook as tqdm
    
all_pred = pd.DataFrame(columns=['EIN', 'text', 'actual_label', 'predicted_label', 'true_pred'])


for i in tqdm(range(len(x_test))):
    prediction = model.predict(np.array([x_test[i]]))    
    text_labels = encoder.classes_ 
    predicted_label = text_labels[np.argmax(prediction[0])]
    text = test_posts.iloc[i]
    ein = trainDF[trainDF['text']==test_posts.iloc[i]]['EIN'].astype(int).drop_duplicates().tolist()
    actual_label = trainDF[trainDF['text']==test_posts.iloc[i]]['label'].drop_duplicates().tolist()
    if(predicted_label == test_tags.iloc[i]):
        true_pred = 'true'
    else:
        true_pred = 'false'

    all_pred.loc[len(all_pred)] = [ein, text, actual_label, predicted_label, true_pred]


if(os.path.exists('../../data/classification_results/all_predictions_V3.pkl.gz')):
    all_pred = pd.concat([pd.read_pickle('../../data/classification_results/all_predictions_V3.pkl.gz'), all_pred]).drop_duplicates()
    
all_pred.to_pickle('../../data/classification_results/all_predictions_V3.pkl.gz')


In [5]:
print(all_pred[all_pred.true_pred=="false"])

                                                      EIN  \
8                                             [205404353]   
20                                            [330724044]   
33                                            [352102110]   
40                                            [911956621]   
42                                            [260237807]   
44                                            [954659261]   
47                                            [510465035]   
51                                            [942581062]   
53                                            [990352548]   
62                                            [311655576]   
78                                            [232835435]   
104                                           [742550748]   
115                                           [200547957]   
118                                           [205704991]   
145                     [382780230, 581486729, 710815304]   
146                     